In [1]:
import torch
import torch.nn as nn

In [31]:
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # For-loop is the same as before: Get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float("-inf")).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next == eos_id:  # Stop generating early if end-of-sequence token is encountered and eos_id is specified
            break

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx

def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [3]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.attention = MultiHeadCasualAttention(
            d_in= cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            dropout=cfg["drop_rate"],
            num_heads=cfg["n_heads"],
            qkv_bias=cfg["qkv_bias"]
        )

        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # This block does nothing and just returns its input.
        shortcut = x
        x = self.norm1(x)
        x = self.attention(x)
        x = self.drop_shortcut(x)

        x = x + shortcut

        shortcut = x

        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        return x


class LayerNorm(nn.Module):
    def __init__(self, emb_dim, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
        

    def forward(self, x):
        # This layer does nothing and just returns its input.

        mean = x.mean(dim = -1, keepdim = True)
        var = x.var(dim=-1,keepdim = True, unbiased = False)
        norm_x = (x-mean)/torch.sqrt(var+self.eps)
        return self.scale * norm_x + self.shift

class MultiHeadCasualAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec



class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device = in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)

        logits = self.out_head(x)
        return logits

# Load Model

In [4]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 512, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [6]:
snapshot_model = GPTModel(GPT_CONFIG_124M)

# /kaggle/input/py-gpt2/pytorch/default/1/gpt2_ddp_model_1.pth

snapshot_model.load_state_dict(torch.load("/home/gv/Downloads/apps/gpt2_ddp_model_2_cpu.pth", weights_only=True))
# snapshot_model.eval()
device = torch.device("cpu")
snapshot_model.to(device)

/home/gv/envs/ml/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(512, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (attention): MultiHeadCasualAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (attention): MultiHeadCasualAttention(
        (W_qu

# Validation

In [10]:
# Custom Dataset (assuming you have this)
import torch
import torch.nn as nn
import torch.optim as optim
import tiktoken
from torch.utils.data import DataLoader, Dataset
import os
class CustomDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    
def create_dataloader_v1(txt, batch_size=8, max_length=256,
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = CustomDataset(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader


def load_train_val_dataset():
    from datasets import load_dataset

    train_dataset = load_dataset('flytech/python-codes-25k', split='train')
    # train_dataset = load_dataset('flytech/python-codes-25k', split='test')

    # One can map the dataset in any way, for the sake of example:
    dataset = train_dataset.map(lambda example: {'text': example['instruction'] + ' ' + example['input'] + ' ' + example['output']})['text']
    # Remember that you don't need to map if the dataset has a "text" field already:)
    train_ratio = 0.90
    # text_data = "<|endoftext|>".join(dataset[:50])
    dataset = dataset[:]
    split_idx = int(train_ratio * len(dataset))

    train_data = dataset[:split_idx]
    val_data = dataset[split_idx:]
    train_data = "<|endoftext|>".join(train_data[:])
    val_data = "<|endoftext|>".join(val_data[:])
    # print("split_idx: ", split_idx)

    torch.manual_seed(123)

    train_loader = create_dataloader_v1(
        train_data,
        batch_size=8,
        max_length=GPT_CONFIG_124M["context_length"],
        stride=GPT_CONFIG_124M["context_length"],
        drop_last=True,
        shuffle=True,
        num_workers=0
    )

    val_loader = create_dataloader_v1(
        val_data,
        batch_size=8,
        max_length=GPT_CONFIG_124M["context_length"],
        stride=GPT_CONFIG_124M["context_length"],
        drop_last=False,
        shuffle=False,
        num_workers=0
    )
    print("length of dataset", len(train_data), len(val_data))
    # print("length of loader", len(train_loader), len(val_loader))
    return train_loader, val_loader

In [11]:

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

def calc_loss_batch(input_batch, target_batch, model, device):
    try:
        input_batch, target_batch = input_batch.to(device), target_batch.to(device)
        with torch.no_grad():
            logits = model(input_batch)
            loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
        del input_batch, target_batch, logits
        torch.cuda.empty_cache()
        return loss
    except Exception as exp:
        print(exp)
        return 0

def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss
        else:
            break
    return total_loss / num_batches

In [12]:
train_loader, val_loader = load_train_val_dataset()

Generating train split:   0%|          | 0/49626 [00:00<?, ? examples/s]

Map:   0%|          | 0/49626 [00:00<?, ? examples/s]

length of dataset 19989246 3200684


In [ ]:
len(val_loader)

In [10]:
# 220 * 512 * 2 * 8 * 4

In [13]:
# from tqdm import tqdm
# total_train_loss = 0
# train_idx = 0
# snapshot_model.eval()

# for input_batch, target_batch in tqdm(train_loader, total = len(train_loader)):  
#     train_loss = calc_loss_batch(input_batch, target_batch, snapshot_model, device=torch.device("cuda"))
#     train_idx += 1
#     total_train_loss += (train_loss.item())
#     del input_batch, target_batch
#     torch.cuda.empty_cache()


# # if val_idx:
# # val_loss = total_val_loss/(val_idx)
# # validation_loss.append(val_loss)
# # print(f'Validation Loss: {val_loss:.4f}')

In [28]:
from tqdm import tqdm

def validate_model(model, val_loader ):

    total_val_loss = 0
    val_idx = 0
    val_loss_list = []
    model.eval()
    
    for input_batch, target_batch in tqdm(val_loader, total = len(val_loader)):
        val_loss = calc_loss_batch(input_batch, target_batch, model, device=torch.device("cpu"))
        val_idx += 1
        total_val_loss += (val_loss)
        val_loss_list.append(val_loss)
        # break

    print("Val loss: ", total_val_loss/val_idx)
    return val_loss_list, total_val_loss/val_idx


In [15]:
# next(iter(train_loader))[0].shape

# Model Size

In [16]:
def calculate_model_size(model):
    """
    Calculates the total size of a PyTorch model in bytes.

    Args:
        model (torch.nn.Module): The PyTorch model.

    Returns:
        int: The total size of the model in bytes.
    """
    total_params = 0
    for param in model.parameters():
        total_params += param.numel() * param.element_size()
    return format_size(total_params)

def format_size(size_bytes):
    """
    Formats a size in bytes into a human-readable format (KB, MB, GB).

    Args:
        size_bytes (int): The size in bytes.

    Returns:
        str: The formatted size.
    """
    if size_bytes < 1024:
        return f"{size_bytes} bytes"
    elif size_bytes < (1024 * 1024):
        return f"{size_bytes / 1024:.2f} KB"
    elif size_bytes < (1024 * 1024 * 1024):
        return f"{size_bytes / (1024 * 1024):.2f} MB"
    else:
        return f"{size_bytes / (1024 * 1024 * 1024):.2f} GB"



# Quantized Model

### Dynamic Quantization

In [17]:
calculate_model_size(snapshot_model)

'620.33 MB'

In [18]:
from torch.quantization import quantize_dynamic
model_quantized = quantize_dynamic(
    model=snapshot_model, qconfig_spec={nn.Linear}, dtype=torch.qint8, inplace=False
)


In [19]:
calculate_model_size(model_quantized)

'148.88 MB'

In [20]:
torch.save(model_quantized.to(torch.device("cpu")).state_dict(), "GPT2_quantize_1.pth")

In [23]:
device = torch.device("cpu")

model_quantized.to(device)
snapshot_model.to(device)
print()

### validation of GPT2 Dynamic Quantization

In [30]:
_, snapshot_loss = validate_model(snapshot_model, val_loader )
print("Base Model Val Loss: ", snapshot_loss)

100%|██████████| 220/220 [47:43<00:00, 13.01s/it] 

Val loss:  tensor(1.3223)
Base Model Val Loss:  tensor(1.3223)


In [29]:
_, quant_model_loss = validate_model(model_quantized, val_loader )
print("Quantized Model Val Loss: ", quant_model_loss)

100%|██████████| 220/220 [38:54<00:00, 10.61s/it]

Val loss:  tensor(1.3350)
Quantized Model Val Loss:  tensor(1.3350)


In [17]:
# snapshot_model.qconfig = torch.ao.quantization.default_qconfig
# net_quantized = torch.ao.quantization.prepare(snapshot_model) # Insert observers
# net_quantized = torch.ao.quantization.convert(net_quantized)


# model_size_bytes = calculate_model_size(net_quantized)
# model_size_formatted = format_size(model_size_bytes)

# print(f"Total size of the model: {model_size_formatted}")

In [32]:
# # Define the quantization configuration
# model_old = snapshot_model
# snapshot_model.to(torch.device("cpu"))
# quantization_config = torch.ao.quantization.get_default_qconfig('fbgemm')
# snapshot_model.config.torch_dtype = torch.float32 # Ensure original dtype is float32

# # Prepare the model for static quantization
# prepared_model = torch.ao.quantization.prepare(snapshot_model, quantization_config)

# # quantized_model = torch.quantization.quantize_dynamic(
# #     snapshot_model.to(torch.device("cpu")), {torch.nn.Linear}, dtype=torch.qint8
# # )

# quantized_model = torch.ao.quantization.convert(prepared_model, inplace=False)

# torch.save(quantized_model.state_dict(), '/kaggle/working/quantized_8b_gpt2_model.pth')

# val_loss_list, val_loss = validate_model(quantized_model.to(torch.device("cuda")), val_loader)
# print(f"Quantized Average Loss: {val_loss}")
# # print(f"Quantized Perplexity: {quantized_perplexity}")

In [ ]:
# quantization_config

In [ ]:
# for name, param in quantized_model.named_parameters():
#     print(f"Parameter name: {name}, Data type: {param.data.dtype}")

In [34]:

def text_to_token_ids(text, tokenizer):
    # encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())


### Generate

In [ ]:
torch.manual_seed(123)
tokenizer = tiktoken.get_encoding("gpt2")

In [38]:
import time
start = time.time()

token_ids = generate(
    model=snapshot_model,
    idx=text_to_token_ids("Python program to calculate the average", tokenizer),
    max_new_tokens=250,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=5,
    temperature=0
)
end = time.time()
print("Time taken: ", end-start)
print("Tokens/s: ", 250/(end-start))
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))


Time taken:  84.72966575622559
Tokens/s:  2.950560441478326
Output text:
 Python program to calculate the average age of a group of people people = [
 {'name': 'John', 'age': 30},
 {'name': 'Jane', 'age': 28},
 {'name': 'Jane', 'age': 28}
]  ```python
def calculate_average_age(grades):
 total = 0
 for record in people:
 total += record['age']
 return total / len(people)

people = [{'name': 'John', 'age': 30},
 {'name': 'Jane', 'age': 25},
 {'name': 'James', 'age': 28}
]

total = calculate_average_age(people)
print(total) # Output: 60
```<|endoftext|>Create a Python program to identify if the given string contains any of the characters such that the string contains at least two numbers: 6, 8  ```python
def is_two_numbers(string):
    """
    This function checks if two strings are anagrams or not
    
    Parameters:
    string (str): the string
   
 


In [39]:
import time
start = time.time()

token_ids = generate(
    model=model_quantized,
    idx=text_to_token_ids("Python program to calculate the average", tokenizer),
    max_new_tokens=250,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=5,
    temperature=0
)
end = time.time()
print("Time taken: ", end-start)
print("Tokens/s: ", 250/(end-start))
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))


Time taken:  62.362414598464966
Tokens/s:  4.0088248925203365
Output text:
 Python program to calculate the average age of a person based on their weight and weight in the school  ```python
def calculate_average_age(weight):
    total_age = 0
    for person in weight:
        total_age += person['age']
    return total_age
```<|endoftext|>Create a basic web scraper in Python to extract information from an e-commerce website. The scraper should be able to extract product names and prices from an e-commerce website  ```python
import requests
from bs4 import BeautifulSoup

# Get the html content
url = 'https://example.com/product'

# Make the response
response = requests.get(url)

# Parse the response
soup = BeautifulSoup(response.text, 'html.parser')

# Get the list of product names and prices
products = soup.find_all('div', {'class': 'product-product-list-price'})

# Loop over the products and print the result
for product in products:
 print(product.text)
```<|endoftext|>Given a list
